In [2]:
import os
import numpy as np
import json
import cv2
from tqdm.notebook import tqdm

# Main Road Data 확인

In [3]:
data_path = "D:/data/도로장애물·표면 인지 영상(수도권)/Training/!CHANGE/CRACK/"
all_annot = data_path.replace("!CHANGE", "Annotations")
all_image = data_path.replace("!CHANGE", "Images")

annot_path = all_annot+"C_Mainroad_B01/"
image_path = all_image+"C_Mainroad_B01/"

images = os.listdir(image_path)
labels = os.listdir(annot_path)
masks = list(map(lambda x: x.replace("_PLINE.json",".png"),labels))
masks[:5]

['V1F_HY_9341_20210114_170155_N_CH0_Seoul_Sun_Mainroad_Day_54453.png',
 'V1F_HY_9342_20210114_165322_E_CH1_Seoul_Sun_Mainroad_Day_16726.png',
 'V1F_HY_9343_20210114_164821_E_CH1_Seoul_Sun_Mainroad_Day_26198.png',
 'V1F_HY_9343_20210115_151928_E_CH1_Seoul_Sun_Mainroad_Day_17453.png',
 'V1F_HY_9344_20160212_020303_N_CH1_Seoul_Sun_Mainroad_Day_94537.png']

In [4]:
set(images)-set(masks)

set()

데이터 자체에 오류는 없는 것으로 보임. 그러면 오류가 날 수 있느 부분은 다음과 같음
- `.json`을 불러올 때 `ndarray`로 변환하는 부분
- `.json`을 불러오고 `float` 데이터를 `int`로 변환하는 부분

# json에서 polyline으로 변환 로직 확인

In [5]:
from pathlib import Path
ap_lst = []
for f_name in tqdm(labels):
    filename=annot_path+f_name
    with open(filename,"r",encoding='utf8') as f:
        contents = f.read()
        json_data = json.loads(contents)

    str_fname = str(filename)
    img_pth = Path(str_fname.replace("Annotations","Images").replace("_PLINE.json",".png"))            
    load_img = np.array(Image.open(img_pth))    
    lbl = np.zeros((load_img.shape[0], load_img.shape[1]), np.int32)

    for idx in range(len(json_data["annotations"])):
        temp = np.array(json_data["annotations"][idx]["polyline"]).reshape(-1)
        
        """
        문제가 발생하는 np.apply_along_axis 라인에서 예외 처리 실시
        
        오류가 발생하면 발생한 file의 이름을 확인
        """
        try:
            temp_int = np.apply_along_axis(np.int32, arr=temp, axis=0)
        except:
            print(filename)
            assert 0!=0, f"In Error"
        temp_re = temp_int.reshape(-1, 2)
        
        lbl = cv2.polylines(img=lbl,
                    pts=[temp_re],
                    isClosed=False,
                    color=(255),
                    thickness=5)
    ap_lst.append(lbl)

  0%|          | 0/10000 [00:00<?, ?it/s]

D:/data/도로장애물·표면 인지 영상(수도권)/Training/Annotations/CRACK/C_Mainroad_B01/V2F_HY_0031_20201202_120432_N_CH0_Seoul_Sun_Mainroad_Day_98665_PLINE.json


AssertionError: In Error

error 탐지 결과 `D:/data/도로장애물·표면 인지 영상(수도권)/Training/Annotations/CRACK/C_Mainroad_B01/V2F_HY_0031_20201202_120432_N_CH0_Seoul_Sun_Mainroad_Day_98665_PLINE.json` 해당 경로 데이터에서 문제가 발생한 것으로 보임. 확인을 해야겠지만, `cv2.polylines()`로 적용할 point가 없을 가능성이 높음

# 오류가 발생한 file의 데이터 값 확인

In [6]:
err_file = "D:/data/도로장애물·표면 인지 영상(수도권)/Training/Annotations/CRACK/C_Mainroad_B01/V2F_HY_0031_20201202_120432_N_CH0_Seoul_Sun_Mainroad_Day_98665_PLINE.json"

with open(err_file,"r",encoding='utf8') as f:
    contents = f.read()
    json_data = json.loads(contents)
    
json_data["annotations"][0]["polyline"][0][:6]

[None,
 None,
 247.1428571428571,
 712.8571428571428,
 244.76190476190476,
 695.7142857142857]

확인 결과 `.json`의 `polyline`에 `None`이 들어가 있음. 따라서 불러오는 단계에서 `None` 좌표 값을 해결해야 함

# None Type 해결 로직 생성

In [7]:
def remove_none(lst):
    if -1 in lst:
        print(lst.remove(-1))
        return remove_none(lst.remove(-1))
    else:
        return lst



t = json_data["annotations"][0]["polyline"]
tmp = [[x for x in t[0] if x is not None]]
temp = np.array(tmp).reshape(-1)
temp_round = np.apply_along_axis(np.round , arr=temp, axis=0)
temp_int = np.apply_along_axis(np.int32 , arr=temp_round, axis=0)
temp_re = temp_int.reshape(-1, 2)
temp_re

array([[247, 713],
       [245, 696],
       [258, 680],
       [282, 666],
       [274, 655],
       [271, 637],
       [270, 627],
       [287, 606],
       [270, 595],
       [277, 589],
       [265, 570],
       [270, 563],
       [268, 549],
       [287, 541],
       [298, 531],
       [304, 528],
       [293, 520],
       [291, 515]])